## Install requirements

In [1]:
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import model and tokenizer

In [1]:
# Select CUDA device index
import os
import torch
#from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

os.environ["CUDA_VISIBLE_DEVICES"] = "0"



model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

## Prepare model for training

In [2]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


## Load your `PeftModel`

LoRA (Low-Rank Adaptators) to train the model

In [3]:
from peft import LoraConfig, get_peft_model, TaskType


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM"
)


model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 1769472 || all params: 249347328 || trainable%: 0.7096414524241463


## Load and process data



In [4]:
import pandas as pd

clean_df = pd.read_json('/content/cleanquest.json')
clean_df

,question,answers,correct_answer,explanation,explanationImageHtml,id,clean_explanation
0,Mountain waves should be expected..,"[{'id': 1, 'answer': 'directly over the mounta...",on the downwind side of the mountain range.,Refer to figure.\nMOUNTAIN/LEE WAVES\ndevelop ...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",72880,".\nMOUNTAIN/LEE WAVES\ndevelop on the lee, or ..."
1,Which cloud poses the highest risk to aviation...,"[{'id': 1, 'answer': 'Cumulus fractus.'}, {'id...",Cumulonimbus.,Refer to figure.\nWhen cold air displaces warm...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",96266,".\nWhen cold air displaces warm air, it create..."
2,"According to ICAO Annex 3 definition, ""visibil...","[{'id': 1, 'answer': '1 is correct, 2 is corre...","1 is correct, 2 is incorrect.","In effect,\nvisibility\nis a measure of atmosp...",,96218,"In effect,\nvisibility\nis a measure of atmosp..."
3,"An aerodrome on the coast (Faro, Portugal) rep...","[{'id': 1, 'answer': 'Humidity reducing becaus...",A cold front crossing the area.,Refer to figure.\nNote: This question has two ...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",80606,.\nNote: This question has two correct answers...
4,Which of the following statements concerning a...,"[{'id': 1, 'answer': 'Air parcels that are for...",An isothermal layer is absolutely stable.,An isothermal layer is defined as a vertical c...,,72780,An isothermal layer is defined as a vertical c...
...,...,...,...,...,...,...,...
1447,A mountain range spans from North to South. A ...,"[{'id': 1, 'answer': 'Cloud base higher than 2...",Cloud base higher than 2 000 ft and colder air...,Refer to figure.\nThis question is a little tr...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",82192,.\nThis question is a little tricky as it is a...
1448,Which type of cloud is most likely to be encou...,"[{'id': 1, 'answer': 'Altocumulus'}, {'id': 2,...",Altocumulus,Refer to figure.\nNote: This question is quite...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",100595,".\nNote: This question is quite challenging, a..."
1449,Advection fog is most likely to form when:,"[{'id': 1, 'answer': 'maritime warm air flows ...",a mild moist airstream flows over snow covered...,Refer to figure.\nADVECTION FOG\n. Is formed b...,"<div class=""mt10 fl wd100""> <div class=""fl tc ...",73249,.\nADVECTION FOG\n. Is formed by the advection...
1450,Immediately following an extended flight at hi...,"[{'id': 1, 'answer': 'Mixed ice on the leading...",Frost forming around the fuel tanks due to col...,"Refer to figure.\nIn this scenario, the aircra...","<div class=""mt10 fl wd100""> <div class=""fl tc ...",82240,".\nIn this scenario, the aircraft has a very c..."


In [5]:
from datasets import Dataset

train=clean_df.head(1000)
test=clean_df.tail(452)

train_dataset = Dataset.from_pandas(train)
val_dataset=Dataset.from_pandas(test)

In [6]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install -q rouge_score
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 22.3 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,TextSplitter,CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [11]:
# LOAD: explanations as the contextt db
list_of_documents=[]
for clean_exp in train.clean_explanation.values:
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=16)

  list_of_documents.extend(text_splitter.split_documents([Document(page_content=clean_exp)]))

In [14]:
for pdf in ["/path_to_book1.pdf","/path_to_book2.pdf"]:
  loader = PyPDFLoader(pdf)
  pages = loader.load_and_split()
  list_of_documents.extend(text_splitter.split_documents(pages))


In [15]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
#modelPath = "sentence-transformers/all-mpnet-base-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
db = FAISS.from_documents(list_of_documents, embeddings)

Filtering step as it was observed some pages consisted of questions which was adding noisy documents as relevant in the retrieval phase of the pipeline

In [17]:
pages_toremove=[15,16,17,18,19, 28, 29, 30, 31, 32, 33,40,41, 54, 55, 56, 57, 58,59,76,77,78,79,88,89, 90,91,102,103,104,105, 106,107,124,125,126,127,128,129,143,144,145,146,147,148,149,150,151,152,153,174,175,176,177,178,179,180,181,191,192,193,194,195,196,211,212,213,214,215,216,217,218,219,230,231,232,234,235,251,252,253,254,270,271,283,284,285,286,287,288,289,301,302,303,304,305,323,324,325,326,327,328,329,330,331,339,340,341,342,343,344,345,346,347,348,366,367,368,369,370,371,397,398,399,400,401,402,403,404,405,406,407,408,409,418,419,486,487,488,489,490,496,497,498,499,500,512,513,514,515,516,517,518,519,520,528,529,530,531,532,533,534,535,536,537,538,539,540,541]+[num for num in range(550,664)]
keys_to_remove=[]
for key in db.docstore._dict.keys():
  if 'page' in db.docstore._dict[key].metadata.keys() and db.docstore._dict[key].metadata['source']=='/content/vdoc.pub_cae-oxford-aviation-academy-atpl-book-9-meteorology.pdf':
    if db.docstore._dict[key].metadata['page'] in pages_toremove:
      keys_to_remove.append(key)
db.delete(keys_to_remove)

True

In [18]:
def preprocess_data(question, answer, max_length=128):
  inputs = tokenizer(
  question,
  max_length=max_length,
  padding='max_length',
  truncation=True,
  return_tensors='pt'
  )
  input_ids = inputs['input_ids'].squeeze()
  attention_mask = inputs['attention_mask'].squeeze()

  # Encode the answer

  # Tokenize targets with text_target=...
  labels = tokenizer(text_target=answer, max_length=max_length, padding='max_length', truncation=True)

  # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
  # padding in the loss.

  labels["input_ids"] = [(l if l != tokenizer.pad_token_id else -100) for l in labels["input_ids"]]

  inputs["labels"] = labels["input_ids"]

  return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': inputs["labels"]

  }

In [19]:
preprocessed_train_dataset = []
preprocessed_val_dataset = []
from tqdm import tqdm
for example in tqdm(train_dataset):
  question=example['question']
  option1=example['answers'][0]['answer']
  option2=example['answers'][1]['answer']
  option3=example['answers'][2]['answer']
  option4=example['answers'][3]['answer']
  context=''
  question="For the question: "+ question +" ,choose the correct answer from the following answers: option1) " + option1 +", option2) "+ option2 +", option3) "+ option3 + ", option4) "+option4
  for docs in db.search(question,search_type='mmr',k=10):
    context+=docs.page_content+'\n'

  question="context: "+context+ " Use the context to answer the following question. Answer using the context only. "+ question
  preprocessed_example = preprocess_data(question, example['correct_answer'])
  preprocessed_train_dataset.append(preprocessed_example)

for example in tqdm(val_dataset):
  question=example['question']
  option1=example['answers'][0]['answer']
  option2=example['answers'][1]['answer']
  option3=example['answers'][2]['answer']
  option4=example['answers'][3]['answer']
  context=''
  question="For the question: "+ question +" ,choose the correct answer from the following answers: option1) " + option1 +", option2) "+ option2 +", option3) "+ option3 + ", option4) "+option4

  for docs in db.search(question,search_type='mmr',k=10):
    context+=docs.page_content+'\n'
  question="context: "+context+ " Use the context to answer the following question. Answer using the context only. "+question
  preprocessed_example = preprocess_data(question, example['correct_answer'])
  preprocessed_val_dataset.append(preprocessed_example)


100%|██████████| 452/452 [00:28<00:00, 15.83it/s]


In [20]:
tokenized_train_dataset = Dataset.from_dict(
{key: [example[key] for example in preprocessed_train_dataset] for key in preprocessed_train_dataset[0].keys()})
print(tokenized_train_dataset)

tokenized_val_dataset = Dataset.from_dict(
{key: [example[key] for example in preprocessed_val_dataset] for key in preprocessed_val_dataset[0].keys()})
print(tokenized_val_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 452
})


Let's also apply some pre-processing of the input data, the labels needs to be pre-processed, the tokens corresponding to `pad_token_id` needs to be set to `-100` so that the `CrossEntropy` loss associated with the model will correctly ignore these tokens.

## Finetuning



In [21]:
from transformers import DataCollatorForSeq2Seq,T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [15]:
import torch
torch.cuda.empty_cache()

In [22]:
import nltk
import evaluate
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")
import numpy as np


In [24]:
# Global Parameters
L_RATE = 15e-6 #3e-4 #15e-6,
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.02
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 8

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./resultss",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   save_strategy='epoch',
   load_best_model_at_end=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
)

In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,No log,3.361646
2,3.353400,3.244041
3,3.353400,3.155964
4,3.157500,3.101147
5,3.157500,3.070980
6,3.083800,3.053898
7,3.083800,3.044508
8,3.060700,3.041618


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/

TrainOutput(global_step=2000, training_loss=3.1638834228515624, metrics={'train_runtime': 1374.4203, 'train_samples_per_second': 5.821, 'train_steps_per_second': 1.455, 'total_flos': 1380386340864000.0, 'train_loss': 3.1638834228515624, 'epoch': 8.0})

In [26]:
last_checkpoint = '/content/resultss/checkpoint-2000'


model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(last_checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [27]:
question_answerer = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.1, "max_length": 512},
)


In [28]:
retriever = db.as_retriever(
     search_kwargs={"k":10 }, search_type = 'mmr'
)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

In [29]:
answers=[]
from tqdm import tqdm
for i in tqdm(range(1000,1452)):
  question=clean_df['question'].iloc[i]
  option1=clean_df['answers'].iloc[i][0]['answer']
  option2=clean_df['answers'].iloc[i][1]['answer']
  option3=clean_df['answers'].iloc[i][2]['answer']
  option4=clean_df['answers'].iloc[i][3]['answer']
  # question="For the question"+ question +"choose the correct answer from the following answers" + option1 +","+ option2 +","+ option3 + ","+option4
  question="For the question: "+ question +" ,choose the correct answer from the following answers: option1) " + option1 +", option2) "+ option2 +", option3) "+ option3 + ", option4) "+option4
  response = qa({"query": question},return_only_outputs=True)
  #print(response['result'])
  answers.append(response['result'].lower().replace(clean_df['question'].iloc[i].lower(),'').strip())

  0%|          | 0/452 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1384: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/452 [00:05<39:25,  5.25s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1384: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 2/452 [00:09<34:36,  4.62s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1384: UserWarning: Using the model-agnostic defa

In [30]:
import pandas as pd

# Step 1: Create an empty DataFrame
columns = ['predicted_answer', 'groundtruth', 'matches?','options']  # Replace with your column names
result_df = pd.DataFrame(columns=columns)

In [ ]:
import string
cnt=0
j=0
for i in range(len(answers)):
  matches=0
  if answers[j].lower().strip().translate(str.maketrans('', '', string.punctuation)).replace('.','').replace('helpful','').replace('help','')==clean_df.tail(452)['correct_answer'].values[i].lower().strip().translate(str.maketrans('', '', string.punctuation)).replace('.',''):
    cnt+=1
    matches=1

  row_data = {'predicted_answer': answers[j], 'groundtruth': clean_df.tail(452)['correct_answer'].values[i], 'matches?': matches, 'options':clean_df.tail(452)['answers'].values[i]}  # Replace with your values
  result_df = result_df.append(row_data, ignore_index=True)
  j+=1
print(str(cnt)+" out of " + str(len(answers))+" are correct")

In [ ]:
result_df

In [32]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
def rouge_l_score(reference, generated):
    smoothing_function = SmoothingFunction().method1  # Choose a smoothing function

    # Tokenize the strings into lists of words
    reference_tokens = reference.split()
    generated_tokens = generated.split()

    # Compute ROUGE-L score
    rouge_l = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)

    return rouge_l


def map_to_option_id_gd(row):
  target_answer=row['groundtruth']
  answer_options=row['options']
  for option in answer_options:
      if target_answer.lower() in option['answer'].lower():
          return int(option['id'])
  return None
def map_to_option_id_pred(row):
  target_answer=row['predicted_answer']
  answer_options=row['options']
  maxrouge=0
  id=None
  for option in answer_options:
      rouge_lscore=rouge_l_score(option['answer'].lower(),target_answer.lower())
      if rouge_lscore > maxrouge:
          maxrouge=rouge_lscore
          id=int(option['id'])
  return id

In [33]:
result_df['result_id_groundtruth'] = result_df.apply(map_to_option_id_gd, axis=1)

In [34]:
result_df['result_id_prediction']=result_df.apply(map_to_option_id_pred, axis=1)

In [35]:
result_df['matches_ids']=result_df.result_id_groundtruth==result_df.result_id_prediction

In [36]:
result_df.matches_ids.value_counts()[True]/len(result_df)  #accuracy of google flan base model after finetuning

0.40707964601769914

In [37]:
result_df.matches_ids.value_counts()

False    268
True     184
Name: matches_ids, dtype: int64

In [38]:
result_df

,predicted_answer,groundtruth,matches?,options,result_id_groundtruth,result_id_prediction,matches_ids
0,valley inversion.,Valley inversion.,1,"[{'id': 1, 'answer': 'Valley inversion.'}, {'i...",1,1.0,True
1,increases exponentially.,Increases exponentially.,1,"[{'id': 1, 'answer': 'Increases linearly betwe...",2,2.0,True
2,surface friction.,Horizontal pressure difference.,0,"[{'id': 1, 'answer': 'Earth rotation.'}, {'id'...",2,3.0,False
3,11 to 50 km.,11 to 50 km.,1,"[{'id': 1, 'answer': '50 to 85 km.'}, {'id': 2...",3,3.0,True
4,snow,Hail,0,"[{'id': 1, 'answer': 'Hail'}, {'id': 2, 'answe...",1,2.0,False
...,...,...,...,...,...,...,...
447,cloud base lower than 2 000 ft and colder air ...,Cloud base higher than 2 000 ft and colder air...,0,"[{'id': 1, 'answer': 'Cloud base higher than 2...",1,2.0,False
448,cumulonimbus,Altocumulus,0,"[{'id': 1, 'answer': 'Altocumulus'}, {'id': 2,...",1,2.0,False
449,maritime warm air flows over a relatively warm...,a mild moist airstream flows over snow covered...,0,"[{'id': 1, 'answer': 'maritime warm air flows ...",2,1.0,False
450,ice crystal icing on the leading edges due to ...,Frost forming around the fuel tanks due to col...,0,"[{'id': 1, 'answer': 'Mixed ice on the leading...",3,2.0,False


# LLM Fine tuning results


| Embedding model | TextSplitter | chunk_size | list_of_documents | LLM model | retriever(top-k) | Accuracy | correct_answers |
|----------|----------|----------|----------|----------|----------|----------|----------|
| all-MiniLM-l6-v2  | Character | 256   | explanation  + book1 + book2   | google/flan-t5-small   |  10  | 0.33  | 149 |
| all-MiniLM-l6-v2  | RecursiveCharacter | 256  | explanation + book1 + book2   | google/flan-t5-base  | 10 | 0.41  | 187  |
| all-MiniLM-l6-v2  | RecursiveCharacter  | 256  | explanation + book1 + book2  | google/flan-t5-large  | 10  | 0.46  | 210 |




> num_epoch = 10

> learning_rate = 15e-6

## Refenence:

1. https://github.com/huggingface/peft